## BBC News Data Preprocessing

In [42]:
import pandas as pd
import os
import glob

from nltk.tokenize import RegexpTokenizer

In [43]:
#Assumes data is in current directory

articles_path = "BBC News Summary/News Articles"
summaries_path = "BBC News Summary/Summaries"

business_articles = articles_path + "/business"
business_summaries = summaries_path + "/business"

entertainment_articles = articles_path + "/entertainment"
entertainment_summaries = summaries_path + "/entertainment"

politics_articles = articles_path + "/politics"
politics_summaries = summaries_path + "/politics"

sport_articles = articles_path + "/sport"
sport_summaries = summaries_path + "/sport"

tech_articles = articles_path + "/tech"
tech_summaries = summaries_path + "/tech"

paths = [[business_articles, business_summaries], 
         [entertainment_articles, entertainment_summaries],
         [politics_articles, politics_summaries],
         [sport_articles, sport_summaries],
         [tech_articles, tech_summaries]]

data = []

for articles, summaries in paths:
    for article, summary in zip(os.listdir(articles), os.listdir(summaries)):
        with(open(articles + "/" + article, "r")) as f1:
            with(open(summaries + "/" + summary, "r")) as f2:
                data.append([f1.read(), f2.read()])
            

In [44]:
df = pd.DataFrame(data, columns=["article", "summary"])

In [45]:
df.head()

,article,summary
0,Ad sales boost Time Warner profit\n\nQuarterly...,TimeWarner said fourth quarter sales rose 2% t...
1,Dollar gains on Greenspan speech\n\nThe dollar...,The dollar has hit its highest level against t...
2,Yukos unit buyer faces loan claim\n\nThe owner...,Yukos' owner Menatep Group says it will ask Ro...
3,High fuel prices hit BA's profits\n\nBritish A...,"Rod Eddington, BA's chief executive, said the ..."
4,Pernod takeover talk lifts Domecq\n\nShares in...,Pernod has reduced the debt it took on to fund...


In [46]:
def clean(sentence):
    sentence = str(sentence).lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    return tokens

In [47]:
clean_articles = []
for entry in df['article']:
    clean_articles.append(clean(entry))

clean_summaries = []
for entry in df['summary']:
    clean_summaries.append(clean(entry))

In [52]:
vocab = list(set([word for article in clean_articles for word in article]
                 + [word for summary in clean_summaries for word in summary]))

In [61]:
word2idx = dict((word, idx) for idx, word in enumerate(vocab))
idx2word = dict((idx, word) for idx, word in enumerate(vocab))